In [9]:
#holay molay :0 ヾ(＠⌒ー⌒＠)ノ  ╚(•⌂•)╝   (≧∀≦)ゞ
#finish:
#min p add ✓
#fix sample nexttoken ✓
#logtoku detection
###adapt get_eu for per-token basis ✓
###implement R_token = -AU_token * EU_token ✓
###logtoku fucntion(create sliding window evaluating r_response = (1/K) Σ R(aₜ) )
#verify generation loop works
#verify hallucination detection works
!pip install transformers
!pip install scipy
!pip install accelerate


[notice] A new release of pip is available: 23.3.1 -> 25.1.1
[notice] To update, run: python -m pip install --upgrade pip
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 25.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.7/37.7 MB 334.6 kB/s eta 0:00:0000:0100:03

[notice] A new release of pip is available: 23.3.1 -> 25.1.1
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 25.1.1
[notice] To update, run: python -m pip install --upgrade pip


In [1]:
import numpy as np
import scipy
from scipy.special import softmax, digamma
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

In [2]:
#Functions for logtoku pasted from their codebase
def topk(arr, k):
    indices = np.argpartition(arr, -k)[-k:]
    values = arr[indices]
    return values, indices
    
def get_uncertainty_metric(logits, mode="eu", k=5):
    if mode == "eu":
        if k is None:
            raise ValueError("k must be provided for 'eu' mode.")

        def eu(logits):
            top_k = k
            if len(logits) < top_k:
                raise ValueError("Logits array length is less than top_k.")
            top_values, _ = topk(logits, top_k)
            mean_scores = top_k / (np.sum(np.maximum(0, top_values)) + top_k)
            return mean_scores
        return eu

    
    elif mode == "au":
        def cal_au(logits):
            top_k = k
            if len(logits) < top_k:
                raise ValueError("Logits array length is less than top_k.")
            top_values = np.partition(logits, -top_k)[-top_k:]
            alpha = np.array([top_values])
            alpha_0 = alpha.sum(axis=1, keepdims=True)
            psi_alpha_k_plus_1 = digamma(alpha + 1)
            psi_alpha_0_plus_1 = digamma(alpha_0 + 1)
            result = - (alpha / alpha_0) * (psi_alpha_k_plus_1 - psi_alpha_0_plus_1)
            return result.sum(axis=1)[0]
        return cal_au
    
    else:
        raise ValueError(f"Unsupported mode: {mode}")

def get_token_reliability(logits, k=25):
    au = get_uncertainty_metric(logits, mode= "au")
    eu = get_uncertainty_metric(logits, mode= "eu", k=25)                            
    return -au * eu


#unused so far?
def get_one_pass_metric(logits, clean_generated_tokens_length, metrics, get_eu, topk):
    """
    Process logits, calculate various metrics, and extract topk information from the logits.
    
    Args:
        
        clean_generated_tokens_length (int): Length of the generated tokens.
        metrics (list): List of metrics and their parameters.
        get_eu (function): Function to retrieve metric values.
        topk (function): Function to extract topk values.
        
    Returns:
        tuple: A tuple containing metric_dict and logit_dict.
    """
    metric_dict = {}

    sequence_length = len(logits)
    for idx_l in range(min(clean_generated_tokens_length, sequence_length)):
        logit = logits[idx_l]
        logit = logit.cpu().numpy()
        
        for metric, k in metrics:
            if metric not in metric_dict:
                metric_dict[metric] = []
            eu = get_eu(metric, k)
            metric_dict[metric].append(eu(logit[0]))

    logit_dict = {}
    logit_start_idx = 0
    logit_end_idx = min(clean_generated_tokens_length, sequence_length)
    ii = 0

    # Extract topk information from logits
    for idx_ll in range(logit_start_idx, logit_end_idx):
        logit = logits[idx_ll]
        logit = logit.cpu().numpy()
        
        top_k = 10
        top_values, top_indices = topk(logit[0], top_k)
        
        logit_dict[ii] = {'top_values': top_values, 'top_indices': top_indices}
        ii += 1

    # Convert top_values and top_indices in logit_dict to list
    for key in logit_dict:
        logit_dict[key]['top_values'] = logit_dict[key]['top_values'].tolist()
        logit_dict[key]['top_indices'] = logit_dict[key]['top_indices'].tolist()

    return metric_dict, logit_dict

In [3]:
class SemanticSpillway:
    def __init__(self, model_path: str = "../models/Llama-3.1-8B-Instruct"):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.tokenizer = AutoTokenizer.from_pretrained(model_path, local_files_only=True)
        self.model = AutoModelForCausalLM.from_pretrained(
            model_path, torch_dtype=torch.float16, local_files_only=True
        ).to(self.device)
        self.reliability_scores = [] #for logtoku

        #not necessary unless/until we do processing in batches(then have to do padding and attention masks and blah blah blah)
        # if self.tokenizer.pad_token is None:
        #     self.tokenizer.pad_token = self.tokenizer.eos_token

    # (currently) uses min p to sample next token
    def sample_next_token(self, logits, temperature=0.7, top_k=50, top_p=0.9, min_p = 0.05, min_tokens_to_keep = 5, do_sample=True):
        
        # Temperature
        logits = logits / temperature
        
        # # top-k 
        # if top_k > 0:
        #     top_k = min(top_k, logits.size(-1))
        #     indices_to_remove = logits < torch.topk(logits, top_k)[0][..., -1, None]
        #     logits[indices_to_remove] = float('-inf')
        
        # # top-p 
        # if top_p < 1.0:
        #     sorted_logits, sorted_indices = torch.sort(logits, descending=True)
        #     cumulative_probs = torch.cumsum(torch.softmax(sorted_logits, dim=-1), dim=-1)
        #     sorted_indices_to_remove = cumulative_probs > top_p
        #     sorted_indices_to_remove[..., 0] = 0
        #     indices_to_remove = sorted_indices_to_remove.scatter(-1, sorted_indices, sorted_indices_to_remove)
        #     logits[indices_to_remove] = float('-inf')

        
        # min-p truncation: gets rid of tokens w/ probabilities less than min_p * max_prob (ie probability of most likely token)
        # pasted from paper
        # turn into methods later i dislike this block of code
        # Convert logits to probabilities
        probs = torch.softmax(logits, dim=-1)
        # Get the probability of the top token for each sequence in the batch
        top_probs, _ = probs.max(dim=-1, keepdim=True)
        # Calculate the actual min_p threshold by scaling min_p with the top token's probability
        scaled_min_p = min_p * top_probs
        # Create a mask for tokens that have a probability less than the scaled min_p
        tokens_to_remove = probs < scaled_min_p

        sorted_indices = torch.argsort(logits, descending=True, dim=-1)
        sorted_indices_to_remove = torch.gather(tokens_to_remove, dim=-1, index=sorted_indices)
        # Keep at least min_tokens_to_keep
        sorted_indices_to_remove[..., : min_tokens_to_keep] = False

        indices_to_remove = sorted_indices_to_remove.scatter(1, sorted_indices, sorted_indices_to_remove)
        filter_value = float('-inf') #TODO verify that this is the correct value to use
        scores_processed = logits.masked_fill(indices_to_remove, filter_value)
        return scores_processed
        
        # sample or greedy
        if do_sample:
            probs = torch.softmax(logits, dim=-1)
            next_token = torch.multinomial(probs, num_samples=1)
        else:
            next_token = torch.argmax(logits, dim=-1, keepdim=True)
            #next_token = torch.argmax(logits, dim=-1).unsqueeze(0)
        return next_token

    # main generation loop w/ stopping control
    # halu_detect = which function to use for hallucination control. currently only option is logtoku, but for regeneration, ner + logtoku function will be created to use instead
    def generation_loop(self, prompt, max_new_tokens=100, temperature=1.0, top_k=50, top_p=0.9, min_p = 0.05, 
                 do_sample=True, halu_detect=None):

        #turn text into token ids
        input_ids = self.tokenizer.encode(prompt, return_tensors="pt").to(self.device)
        input_ids = input_ids.long()
        
        for step in range(max_new_tokens):
            with torch.no_grad(): #.no_grad to ignore gradients for inference
                
                outputs = self.model(input_ids.long())
                logits = outputs.logits[:, -1, :]
                
                # sample next token
                next_token = self.sample_next_token(logits, temperature,  do_sample)
                
                # # check for end of sequence -- if it is, end loop
                # if next_token.item() == self.tokenizer.eos_token_id:
                #     break
                
                # add new token to sequence
                input_ids = torch.cat([input_ids, next_token], dim=-1) 
                
                # callback fn to implement hallucination detection for stopping generation
                # if fn exists and returns false (aka hallucination is detected), stop generating 
                if halu_detect and not halu_detect(step, next_token.item(), input_ids):
                    break
                    
        #turn token ids back into clean text
        return self.tokenizer.decode(input_ids[0], skip_special_tokens=True)


# returns true if no hallucination detected (clean), returns false if not
# used as callback in greater generation loop (intended for initial generation)
def logtoku(step, logits, token_id, input_ids, k=5):
    #empty
    return True

# returns true if no hallucination detected (clean), returns false if not
# used as callback in greater generation loop (intended for comparing with retrieved context)
def ner_based():
    #empty function to fill in later
    return True

#combining the above methods to use at once?!（⊙ｏ⊙）
def ner_logtoku():
    #empty
    return True



generator = SemanticSpillway("../models/Llama-3.1-8B-Instruct")

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [4]:
import torch
import gc

# Check current GPU memory usage
print(f"GPU memory allocated: {torch.cuda.memory_allocated() / 1024**3:.2f} GB")
print(f"GPU memory cached: {torch.cuda.memory_reserved() / 1024**3:.2f} GB")
print(f"GPU memory available: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")

# See what's taking up memory
torch.cuda.memory_summary()

GPU memory allocated: 14.96 GB
GPU memory cached: 15.08 GB
GPU memory available: 23.54 GB


'|===========================================================================|\n|                  PyTorch CUDA memory summary, device ID 0                 |\n|---------------------------------------------------------------------------|\n|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |\n|===========================================================================|\n|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |\n|---------------------------------------------------------------------------|\n| Allocated memory      |  15316 MiB |  15316 MiB |  15316 MiB |      0 B   |\n|       from large pool |  15316 MiB |  15316 MiB |  15316 MiB |      0 B   |\n|       from small pool |      0 MiB |      0 MiB |      0 MiB |      0 B   |\n|---------------------------------------------------------------------------|\n| Active memory         |  15316 MiB |  15316 MiB |  15316 MiB |      0 B   |\n|       from large pool |  15316 MiB |  15316 MiB |

In [6]:
# test generation
result = generator.generation_loop("Once upon a time, Suzy ", max_new_tokens=10, temperature=1.2, do_sample=False)
print(result)

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
# test generation with question to trigger hallucination + logtoku detection
result = generator.generation_loop(
    "What were the main outcomes of the 1847 Treaty of Millbrook between the United States and the Cherokee Nation?", #this doesn't exist btw lol
    max_new_tokens=120,
    halu_detect=logtoku
)
print(result)